# Change Detection (CD) with Random Forest (RF)

### Post-Classification CD using RF classifier
### Using Google Earth Engine Python API and NICFI Normalized Analytic Basemap from December 2015, December 2017, December 2019, and December 2021

Author: Finn Geiger\
Date: April 6th 2023\
Contact:
- https://github.com/finn-geiger
- https://www.linkedin.com/in/finn-geiger-b1329a20b/

### Steps:
1. Importing the datasets and classification result of 2015
2. OPIT classification for 2017, 2019, and 2021
3. CD for TOI 2015 to 2017
4. CD for TOI 2017 to 2019
5. CD for TOI 2019 to 2021
6. Final products

### 1 Import and setup
#### 1.1 Importing the required libraries and packages

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import geemap
import ee
import os
import time
import pandas as pd
from tabulate import tabulate
#%pip install tabulate


The following classes and landcover IDs will be used:

In [2]:
info = {'Class name': ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies'],
        'landcover ID': [1, 2, 3, 4, 5, 6, 7]}

print(tabulate(info, headers='keys', tablefmt='fancy_grid'))

╒══════════════╤════════════════╕
│ Class name   │   landcover ID │
╞══════════════╪════════════════╡
│ Informal     │              1 │
├──────────────┼────────────────┤
│ Formal       │              2 │
├──────────────┼────────────────┤
│ Industrial   │              3 │
├──────────────┼────────────────┤
│ Roads        │              4 │
├──────────────┼────────────────┤
│ Vacant land  │              5 │
├──────────────┼────────────────┤
│ Vegetation   │              6 │
├──────────────┼────────────────┤
│ Water-bodies │              7 │
╘══════════════╧════════════════╛


##### When first using the GEE Python API the user must authenticate and initialize the environment by using the following two lines of codes:

In [3]:
#ee.Authenticate() 
#ee.Initialize()

In [4]:
# creating the map
Map = geemap.Map()

# loading the interactive map
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

#### 1.2 Importing the datasets from GEE assets and data catalog and clipping the basemap to the AOI

In [5]:
# Loading the Base scene
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa')

# Filter basemaps by date and get the first image from filtered results
basemap_2017_12 = nicfi.filter(ee.Filter.date('2017-12-01','2018-01-01')).first()
basemap_2019_12 = nicfi.filter(ee.Filter.date('2019-12-01','2029-01-01')).first()
basemap_2021_12 = nicfi.filter(ee.Filter.date('2021-12-01','2022-01-01')).first()

# importing the classified scene from 2015
classified_TOI1_2015 = ee.Image('users/s85315/masterthesis/change_detection_results/classified_TOI1_RF')

# Visualizing the scene
vis_params = {"bands":["R","G","B"],"min":64,"max":5454,"gamma":1.8}

# Adding the basemap to the map
Map.centerObject(basemap_2017_12, 4)
# Map.addLayer(basemap_2017_12, vis_params, '2017-12 mosaic', False)
# Map.addLayer(basemap_2019_12, vis_params, '2019-12 mosaic', False)
# Map.addLayer(basemap_2021_12, vis_params, '2021-12 mosaic', False)

In [6]:
# Loading the AOI and Masking the base scene
vis_params_aoi = {'color': 'blue'}
aoi_windhoek = ee.FeatureCollection('users/s85315/masterthesis/Study_Area_Windhoek')

# converting the FeatureCollection to Geometry for export
aoi_geom = aoi_windhoek.geometry()

# Adding the AOI to the map
Map.addLayer(aoi_windhoek, vis_params_aoi, 'AOI')
Map.centerObject(aoi_windhoek, 12)

In [7]:
# clipping the clipped_TOI1_2015 to the AOI
clipped_TOI2_2017 = basemap_2017_12.clipToCollection(aoi_windhoek)
clipped_TOI3_2019 = basemap_2019_12.clipToCollection(aoi_windhoek)
clipped_TOI4_2021 = basemap_2021_12.clipToCollection(aoi_windhoek)

Map.addLayer(clipped_TOI2_2017, vis_params, 'clipped_2017', False)
Map.addLayer(clipped_TOI3_2019, vis_params, 'clipped_2019', False)
Map.addLayer(clipped_TOI4_2021, vis_params, 'clipped_2021', False)

In [8]:
# importing training data samples and adding them to the map
# Transforming the Geometries into FeatureCollections and applying properties
training_samples_2017 = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/ChangeDetection/TS_TOI2_2017_RPoints')
training_samples_2019 = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/ChangeDetection/TS_TOI3_2019_RPoints')
training_samples_2021 = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/ChangeDetection/TS_TOI4_2021_RPoints')

# adding the layers to the map
# Map.addLayer(training_samples_2017, {}, 'Training Samples for 2017')
# Map.addLayer(training_samples_2019, {}, 'Training Samples for 2019')
# Map.addLayer(training_samples_2021, {}, 'Training Samples for 2021')

### 2 OPIT Classification with RF for TOIs 2017, 2019, and 2021

#### 2.1 Configuration and creation of the empty RF classifier

In [9]:
# creating variable for parameter settings
# Mtry will be set to default
RF_param = 550

classifier_params = {
              'numberOfTrees':RF_param, # 	Ntree; The number of decision trees to create.
              'variablesPerSplit':None, # Mtry; The number of variables per split. If unspecified, uses the square root of the number of variables.
              'minLeafPopulation':1, # smallest sample size possible per leaf
              'bagFraction':0.5, #The fraction of input to bag per tree.
              'maxNodes':None, # the number of individual decision tree models
              'seed': 0}  # The randomization seed.

#### 2.2 Applying training samples on the scenes

##### 2.2.1 TOI2 - 2017

In [10]:
# adding the training samples to the basescene
training_2017 = clipped_TOI2_2017.sampleRegions(**{
    'collection': training_samples_2017, 
    'properties': ['landcover'], 
    'scale': 4.77,
    'projection': 'EPSG:32733'
})

# creating the classifier using RF
classifier_2017 = ee.Classifier.smileRandomForest(**classifier_params).train(**{
  'features': training_2017,  
  'classProperty': 'landcover', 
  'inputProperties': clipped_TOI2_2017.bandNames()
})

##### 2.2.2 TOI3 - 2019

In [11]:
# adding the training samples to the basescene
training_2019 = clipped_TOI3_2019.sampleRegions(**{
    'collection': training_samples_2019, 
    'properties': ['landcover'], 
    'projection': 'EPSG:32733',
    'scale': 4.77
})

# creating the classifier using RF
classifier_2019 = ee.Classifier.smileRandomForest(**classifier_params).train(**{
  'features': training_2019,  
  'classProperty': 'landcover', 
  'inputProperties': clipped_TOI3_2019.bandNames()
})

##### 2.2.3 TOI4 - 2021

In [12]:
# adding the training samples to the basescene
training_2021 = clipped_TOI4_2021.sampleRegions(**{
    'collection': training_samples_2021, 
    'properties': ['landcover'], 
    'scale': 4.77,
    'projection': 'EPSG:32733'
})

# creating the classifier using RF
classifier_2021 = ee.Classifier.smileRandomForest(**classifier_params).train(**{
  'features': training_2021,  
  'classProperty': 'landcover', 
  'inputProperties': clipped_TOI4_2021.bandNames()
})

#### 2.3 classifying the basescene and visualizing the product

In [13]:
# classifying the basescene using the created classifier
classified_TOI2_2017 = clipped_TOI2_2017.classify(classifier_2017)
classified_TOI3_2019 = clipped_TOI3_2019.classify(classifier_2019)
classified_TOI4_2021 = clipped_TOI4_2021.classify(classifier_2021)

# creating the visualization parameters
palette = ['c43c39', 'e5b636', '2f2f2f', 'aaaaaa', 'b08e7a', '85b66f', 'a5bfdd']
vis_params_classified = {'min': 1, 'max': 7, 'palette': palette}


# Map.addLayer(classified_TOI1_2015, vis_params_classified, 'classified scene - 2015')
# Map.addLayer(classified_TOI2_2017, vis_params_classified, 'classified scene - 2017')
# Map.addLayer(classified_TOI3_2019, vis_params_classified, 'classified scene - 2019')
# Map.addLayer(classified_TOI4_2021, vis_params_classified, 'classified scene - 2021')


#### Calulation of the class size from pixel size

##### OPIT classification for 2017

In [14]:
# calculating pixel size
area_classified_2017 = ee.Image.pixelArea().addBands(classified_TOI2_2017)

# calculating the area size per class
areas_2017 = area_classified_2017.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
    'groupField': 1,
    'groupName': 'landcover',
    }),
    'geometry': aoi_geom, 
    'scale': 4.77,
    'crs': 'EPSG:32733'
    })

# selecting the size per class
class_areas_2017 = ee.List(areas_2017.get('groups'))
list_dict_2017 = class_areas_2017.getInfo()

# creating a dataframe for easier data handling
df_areas_2017 = pd.DataFrame(class_areas_2017.getInfo())
df_areas_2017['sum'] = df_areas_2017['sum'].astype(int)
print(df_areas_2017['sum'].sum())
df_areas_2017.to_csv('./data/class_size_2017.csv', sep=";", index=False)

print(tabulate(df_areas_2017, tablefmt='fancy_grid', headers=['Landcover', 'Area size [m²]'], showindex=False, floatfmt=".0f"))

81696775
╒═════════════╤══════════════════╕
│   Landcover │   Area size [m²] │
╞═════════════╪══════════════════╡
│           1 │         10738343 │
├─────────────┼──────────────────┤
│           2 │          7179838 │
├─────────────┼──────────────────┤
│           3 │          2579538 │
├─────────────┼──────────────────┤
│           4 │          8199046 │
├─────────────┼──────────────────┤
│           5 │         36276700 │
├─────────────┼──────────────────┤
│           6 │         16022588 │
├─────────────┼──────────────────┤
│           7 │           700722 │
╘═════════════╧══════════════════╛


##### OPIT classification for 2019

In [15]:
# calculating pixel size
area_classified_2019 = ee.Image.pixelArea().addBands(classified_TOI3_2019)

# calculating the area size per class
areas_2019 = area_classified_2019.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
    'groupField': 1,
    'groupName': 'landcover',
    }),
    'geometry': aoi_geom, 
    'scale': 4.77,
    'crs': 'EPSG:32733'
    })

# selecting the size per class
class_areas_2019 = ee.List(areas_2019.get('groups'))
list_dict_2019 = class_areas_2019.getInfo()

# creating a dataframe for easier data handling
df_areas_2019 = pd.DataFrame(class_areas_2019.getInfo())
df_areas_2019['sum'] = df_areas_2019['sum'].astype(int)
print(df_areas_2019['sum'].sum())
df_areas_2019.to_csv('./data/class_size_2019.csv', sep=";", index=False)

print(tabulate(df_areas_2019, tablefmt='fancy_grid', headers=['Landcover', 'Area size [m²]'], showindex=False, floatfmt=".0f"))

81696777
╒═════════════╤══════════════════╕
│   Landcover │   Area size [m²] │
╞═════════════╪══════════════════╡
│           1 │          8124641 │
├─────────────┼──────────────────┤
│           2 │          6204410 │
├─────────────┼──────────────────┤
│           3 │          3220814 │
├─────────────┼──────────────────┤
│           4 │         14119783 │
├─────────────┼──────────────────┤
│           5 │         32242821 │
├─────────────┼──────────────────┤
│           6 │         16881483 │
├─────────────┼──────────────────┤
│           7 │           902825 │
╘═════════════╧══════════════════╛


##### OPIT classification for 2021

In [16]:
# calculating pixel size
area_classified_2021 = ee.Image.pixelArea().addBands(classified_TOI4_2021)

# calculating the area size per class
areas_2021 = area_classified_2021.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
    'groupField': 1,
    'groupName': 'landcover',
    }),
    'geometry': aoi_geom, 
    'scale': 4.77,
    'crs': 'EPSG:32733'
    })

# selecting the size per class
class_areas_2021 = ee.List(areas_2021.get('groups'))
list_dict_2021 = class_areas_2021.getInfo()

# creating a dataframe for easier data handling
df_areas_2021 = pd.DataFrame(class_areas_2021.getInfo())
df_areas_2021['sum'] = df_areas_2021['sum'].astype(int)
print(df_areas_2021['sum'].sum())
df_areas_2021.to_csv('./data/class_size_2021.csv', sep=";", index=False)

print(tabulate(df_areas_2021, tablefmt='fancy_grid', headers=['Landcover', 'Area size [m²]'], showindex=False, floatfmt=".0f"))

81696776
╒═════════════╤══════════════════╕
│   Landcover │   Area size [m²] │
╞═════════════╪══════════════════╡
│           1 │         10670301 │
├─────────────┼──────────────────┤
│           2 │          5839601 │
├─────────────┼──────────────────┤
│           3 │          2543647 │
├─────────────┼──────────────────┤
│           4 │          8609501 │
├─────────────┼──────────────────┤
│           5 │         44721504 │
├─────────────┼──────────────────┤
│           6 │          8495030 │
├─────────────┼──────────────────┤
│           7 │           817192 │
╘═════════════╧══════════════════╛


#### 2.4 Exporting the results

##### 2.4.1 Exporting to Google Drive

In [17]:
# # exporting to Google drive with GEE API
# export_TOI2 = ee.batch.Export.image.toDrive(**{
#     'image': classified_TOI2_2017,
#     'description': 'classified_TOI2_RF_test', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'crs': 'EPSG:32733',
#     'scale': 4.77,
#     'region': aoi_geom
# })

# # starting the process
# export_TOI2.start()

# # tracking the process
# while export_TOI2.active():
#   print('Polling for task (id: {}).'.format(export_TOI2.id))
#   time.sleep(5)

In [18]:
# # exporting to Google drive with GEE API
# export_TOI3 = ee.batch.Export.image.toDrive(**{
#     'image': classified_TOI3_2019,
#     'description': 'classified_TOI3_RF_test', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'scale': 4.77,
#     'crs': 'EPSG:32733',
#     'region': aoi_geom
# })

# # starting the process
# export_TOI3.start()

# # tracking the process
# while export_TOI3.active():
#   print('Polling for task (id: {}).'.format(export_TOI3.id))
#   time.sleep(5)

In [19]:
# # exporting to Google drive with GEE API
# export_TOI4 = ee.batch.Export.image.toDrive(**{
#     'image': classified_TOI4_2021,
#     'description': 'classified_TOI4_RF', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'scale': 4.77,
#     'crs': 'EPSG:32733',
#     'region': aoi_geom
# })

# # starting the process
# export_TOI4.start()

# # tracking the process
# while export_TOI4.active():
#   print('Polling for task (id: {}).'.format(export_TOI4.id))
#   time.sleep(5)

##### 2.4.2 Exporting to Asset

In [20]:
# # exporting to Google Asset
# export_TOI2 = ee.batch.Export.image.toAsset(**{
#   'image': classified_TOI2_2017,
#   'description': 'Export classified map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/classified_TOI2_RF', # TODO: change name here
#   'scale': 4.77,
#   'crs': 'EPSG:32733',
#   'region': aoi_geom
# })

# # starting the process
# export_TOI2.start()

# # tracking the process
# while export_TOI2.active():
#   print('Polling for task (id: {}).'.format(export_TOI2.id))
#   time.sleep(5)

In [21]:
# # exporting to Google Asset
# export_TOI3 = ee.batch.Export.image.toAsset(**{
#   'image': classified_TOI3_2019,
#   'description': 'Export classified map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/classified_TOI3_RF', # TODO: change name here
#   'scale': 4.77,
#   'crs': 'EPSG:32733',
#   'region': aoi_geom
# })

# # starting the process
# export_TOI3.start()

# # tracking the process
# while export_TOI3.active():
#   print('Polling for task (id: {}).'.format(export_TOI3.id))
#   time.sleep(5)

In [22]:
# # exporting to Google Asset
# export_TOI4 = ee.batch.Export.image.toAsset(**{
#   'image': classified_TOI4_2021,
#   'description': 'Export classified map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/classified_TOI4_RF', # TODO: change name here
#   'scale': 4.77,
#   'crs': 'EPSG:32733',
#   'region': aoi_geom
# })

# # starting the process
# export_TOI4.start()

# # tracking the process
# while export_TOI4.active():
#   print('Polling for task (id: {}).'.format(export_TOI4.id))
#   time.sleep(5)

### 3 CD for TOI 2015 to 2017

#### 3.1 Performing the CD analysis

In [23]:
changed_areas_2015_2017 = classified_TOI2_2017.subtract(classified_TOI1_2015).neq(0)
changed_areas_2017_2019 = classified_TOI3_2019.subtract(classified_TOI2_2017).neq(0)
changed_areas_2019_2021 = classified_TOI4_2021.subtract(classified_TOI3_2019).neq(0)
changed_areas_2015_2021 = classified_TOI4_2021.subtract(classified_TOI1_2015).neq(0)

# changed_areas_2015_2017.getInfo()

In [24]:
vis_params_changed = {'min':0, 'max':1, 'palette': ['white', 'red']}


Map.addLayer(changed_areas_2015_2017, vis_params_changed, 'Changed areas 2015 to 2017')
Map.addLayer(changed_areas_2017_2019, vis_params_changed, 'Changed areas 2017 to 2019')
Map.addLayer(changed_areas_2019_2021, vis_params_changed, 'Changed areas 2019 to 2021')
Map.addLayer(changed_areas_2015_2021, vis_params_changed, 'Changed areas 2015 to 2021')

transition_2015_2017 = classified_TOI1_2015.multiply(100).add(classified_TOI2_2017).rename('transitions')
transition_2017_2019 = classified_TOI2_2017.multiply(100).add(classified_TOI3_2019).rename('transitions')
transition_2019_2021 = classified_TOI3_2019.multiply(100).add(classified_TOI4_2021).rename('transitions')
transition_2015_2021 = classified_TOI1_2015.multiply(100).add(classified_TOI4_2021).rename('transitions')



#### Calculation of transitions in Pixels

#### Transition for 2015 to 2017

In [25]:
# calculating the size per 
transition_matrix_CD1 = transition_2015_2017.reduceRegion(**{
  'reducer': ee.Reducer.frequencyHistogram(),
  'geometry': aoi_geom,
  'scale': 4.77,
  'tileScale': 16,
  'maxPixels': 1e10,
  'crs': 'EPSG:32733'
  })

transitions_2015_2017 = transition_matrix_CD1.get('transitions').getInfo()
df_transitions_2015_2017 = pd.DataFrame(transitions_2015_2017, index=[0])
df_transitions_2015_2017_long = pd.melt(df_transitions_2015_2017, var_name='From-To', value_name="Pixel")
df_transitions_2015_2017_long['Pixel'] = df_transitions_2015_2017_long['Pixel'].astype(int)


print(df_transitions_2015_2017_long.head())
df_transitions_2015_2017_long.to_csv('./data/transitions_pixels_2015_2017.csv', index=False, sep=";")


  From-To   Pixel
0     101  147445
1     102   54335
2     103   34905
3     104   70090
4     105   43585


#### Calculating the transition values per class for 2015-2017

In [26]:
area_changed_CD1_sqm = ee.Image.pixelArea().addBands(transition_2015_2017)
area_changed_CD1_ha = ee.Image.pixelArea().divide(10000).addBands(transition_2015_2017)


# getting the values per class
areas_changed_CD1_sqm_class = area_changed_CD1_sqm.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
    'groupField': 1,
    'groupName': 'transitions',
    }),
    'geometry': aoi_geom, 
    'scale': 4.77,
    'crs': 'EPSG:32733'
    # 'tileScale': 16,
    # 'maxPixels': 1e10
    })

class_areas = ee.List(areas_changed_CD1_sqm_class.get('groups'))
print(class_areas.getInfo())

df_class_areas = pd.DataFrame(class_areas.getInfo())
df_class_areas['sum'] = df_class_areas['sum'].astype(int)
print(df_class_areas['sum'].sum())
df_class_areas.to_csv('./data/transitions_sqm_per_class_2015_2017.csv', sep=";", index=False)

[{'sum': 3353856.099991952, 'transitions': 101}, {'sum': 1235933.4618437225, 'transitions': 102}, {'sum': 793979.0933117586, 'transitions': 103}, {'sum': 1594304.1067416023, 'transitions': 104}, {'sum': 991403.6023939545, 'transitions': 105}, {'sum': 92627.32020820916, 'transitions': 106}, {'sum': 1786522.2957722233, 'transitions': 201}, {'sum': 2379551.15460327, 'transitions': 202}, {'sum': 732745.3337256189, 'transitions': 203}, {'sum': 1771584.8654028275, 'transitions': 204}, {'sum': 845251.2170141258, 'transitions': 205}, {'sum': 113254.32871189492, 'transitions': 206}, {'sum': 159.21924018859863, 'transitions': 207}, {'sum': 650839.1011419483, 'transitions': 301}, {'sum': 916254.107380422, 'transitions': 302}, {'sum': 717612.7508690329, 'transitions': 303}, {'sum': 469923.19414748397, 'transitions': 304}, {'sum': 34460.27812576294, 'transitions': 305}, {'sum': 7210.567937850952, 'transitions': 306}, {'sum': 90.98426818847656, 'transitions': 307}, {'sum': 1690968.6537452023, 'trans

In [28]:
# calculating the size per 
transition_matrix_CD2 = transition_2017_2019.reduceRegion(**{
  'reducer': ee.Reducer.frequencyHistogram(),
  'geometry': aoi_geom,
  'scale': 4.77,
  'tileScale': 16,
  'maxPixels': 1e10,
  'crs': 'EPSG:32733'
  })

transitions_2017_2019 = transition_matrix_CD1.get('transitions').getInfo()
df_transitions_2017_2019 = pd.DataFrame(transitions_2017_2019, index=[0])
df_transitions_2017_2019_long = pd.melt(df_transitions_2017_2019, var_name='From-To', value_name="Pixel")
df_transitions_2017_2019_long['Pixel'] = df_transitions_2017_2019_long['Pixel'].astype(int)


print(df_transitions_2017_2019_long.head())
df_transitions_2017_2019_long.to_csv('./data/transitions_pixels_2017_2019.csv', index=False, sep=";")


  From-To   Pixel
0     101  147445
1     102   54335
2     103   34905
3     104   70090
4     105   43585


In [29]:
area_changed_CD2_sqm = ee.Image.pixelArea().addBands(transition_2017_2019)
area_changed_CD2_ha = ee.Image.pixelArea().divide(10000).addBands(transition_2017_2019)


# getting the values per class
areas_changed_CD2_sqm_class = area_changed_CD2_sqm.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
    'groupField': 1,
    'groupName': 'transitions',
    }),
    'geometry': aoi_geom, 
    'scale': 4.77,
    'crs': 'EPSG:32733',
    'tileScale': 16,
    'maxPixels': 1e10
    })

class_areas = ee.List(areas_changed_CD2_sqm_class.get('groups'))
print(class_areas.getInfo())

df_class_areas = pd.DataFrame(class_areas.getInfo())
df_class_areas['sum'] = df_class_areas['sum'].astype(int)
print(df_class_areas['sum'].sum())
df_class_areas.to_csv('./data/transitions_sqm_per_class_2017_2019.csv', sep=";", index=False)

[{'sum': 4030994.901947515, 'transitions': 101}, {'sum': 856272.8950868868, 'transitions': 102}, {'sum': 810769.0486038283, 'transitions': 103}, {'sum': 2899439.2973354487, 'transitions': 104}, {'sum': 1973279.4406526454, 'transitions': 105}, {'sum': 167360.48219146728, 'transitions': 106}, {'sum': 227.4703598022461, 'transitions': 107}, {'sum': 751387.9852610644, 'transitions': 201}, {'sum': 3182524.769529021, 'transitions': 202}, {'sum': 647825.6418154623, 'transitions': 203}, {'sum': 1811154.6130795197, 'transitions': 204}, {'sum': 686035.8258737302, 'transitions': 205}, {'sum': 100500.5552870208, 'transitions': 206}, {'sum': 409.44758224487305, 'transitions': 207}, {'sum': 615697.7563844793, 'transitions': 301}, {'sum': 738222.2096057741, 'transitions': 302}, {'sum': 895784.0366549398, 'transitions': 303}, {'sum': 316413.73357229045, 'transitions': 304}, {'sum': 11964.715456008911, 'transitions': 305}, {'sum': 1342.0601768493652, 'transitions': 306}, {'sum': 113.72953033447266, 'tr

In [30]:
# calculating the size per 
transition_matrix_CD3 = transition_2019_2021.reduceRegion(**{
  'reducer': ee.Reducer.frequencyHistogram(),
  'geometry': aoi_geom,
  'scale': 4.77,
  'tileScale': 16,
  'maxPixels': 1e10
  })

transitions_2019_2021 = transition_matrix_CD1.get('transitions').getInfo()
df_transitions_2019_2021 = pd.DataFrame(transitions_2019_2021, index=[0])
df_transitions_2019_2021_long = pd.melt(df_transitions_2019_2021, var_name='From-To', value_name="Pixel")
df_transitions_2019_2021_long['Pixel'] = df_transitions_2019_2021_long['Pixel'].astype(int)

print(df_transitions_2019_2021_long.head())
df_transitions_2019_2021_long.to_csv('./data/transitions_pixels_2019_2021.csv', index=False, sep=";")


  From-To   Pixel
0     101  147445
1     102   54335
2     103   34905
3     104   70090
4     105   43585


In [31]:
area_changed_CD3_sqm = ee.Image.pixelArea().addBands(transition_2019_2021)
area_changed_CD3_ha = ee.Image.pixelArea().divide(10000).addBands(transition_2019_2021)


# getting the values per class
areas_changed_CD3_sqm_class = area_changed_CD3_sqm.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
    'groupField': 1,
    'groupName': 'transitions',
    }),
    'geometry': aoi_geom, 
    'scale': 4.77,
    'crs': 'EPSG:32733',
    'tileScale': 16,
    'maxPixels': 1e10
    })

class_areas = ee.List(areas_changed_CD3_sqm_class.get('groups'))
print(class_areas.getInfo())

df_class_areas = pd.DataFrame(class_areas.getInfo())
df_class_areas['sum'] = df_class_areas['sum'].astype(int)
print(df_class_areas['sum'].sum())
df_class_areas.to_csv('./data/transitions_sqm_per_class_2019_2021.csv', sep=";", index=False)

[{'sum': 3988117.0130767147, 'transitions': 101}, {'sum': 983411.4149369932, 'transitions': 102}, {'sum': 480118.61289343744, 'transitions': 103}, {'sum': 1419632.1324560652, 'transitions': 104}, {'sum': 1195697.5092193305, 'transitions': 105}, {'sum': 57664.435903137804, 'transitions': 106}, {'sum': 1522917.9825707306, 'transitions': 201}, {'sum': 2900949.9154788223, 'transitions': 202}, {'sum': 802201.6037497501, 'transitions': 203}, {'sum': 805812.371485482, 'transitions': 204}, {'sum': 167789.02837834827, 'transitions': 205}, {'sum': 4739.744430803785, 'transitions': 206}, {'sum': 980808.754667312, 'transitions': 301}, {'sum': 712814.3106835908, 'transitions': 302}, {'sum': 699276.3701369043, 'transitions': 303}, {'sum': 507987.2750725316, 'transitions': 304}, {'sum': 281737.44060843973, 'transitions': 305}, {'sum': 22973.35399055481, 'transitions': 306}, {'sum': 15216.736364364624, 'transitions': 307}, {'sum': 3284613.622054208, 'transitions': 401}, {'sum': 1146417.784442004, 'tra

#### Exporting the change mask to Drive and Asset

In [ ]:
# # exporting to Google drive with GEE API
# export_change_mask = ee.batch.Export.image.toDrive(**{
#     'image': changed_areas_2015_2017,
#     'description': 'change_mask_2015_2017', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'scale': 4.77,
#     'crs': 'EPSG:32733',a
#     'region': aoi_geom
# })

# # starting the process
# export_change_mask.start()

# # tracking the process
# while export_change_mask.active():
#   print('Polling for task (id: {}).'.format(export_change_mask.id))
#   time.sleep(5)

In [ ]:
# # exporting to Google drive with GEE API
# export_change_mask = ee.batch.Export.image.toDrive(**{
#     'image': changed_areas_2017_2019,
#     'description': 'change_mask_2017_2019', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'scale': 4.77,
#     'crs': 'EPSG:32733',
#     'region': aoi_geom
# })

# # starting the process
# export_change_mask.start()

# # tracking the process
# while export_change_mask.active():
#   print('Polling for task (id: {}).'.format(export_change_mask.id))
#   time.sleep(5)

In [ ]:
# # exporting to Google drive with GEE API
# export_change_mask = ee.batch.Export.image.toDrive(**{
#     'image': changed_areas_2019_2021,
#     'description': 'change_mask_2019_2021', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'scale': 4.77,
#     'crs': 'EPSG:32733',
#     'region': aoi_geom
# })

# # starting the process
# export_change_mask.start()

# # tracking the process
# while export_change_mask.active():
#   print('Polling for task (id: {}).'.format(export_change_mask.id))
#   time.sleep(5)

In [ ]:
# # exporting to Google drive with GEE API
# export_change_mask = ee.batch.Export.image.toDrive(**{
#     'image': changed_areas_2015_2021,
#     'description': 'change_mask_2015_2021', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'scale': 4.77,
#     'crs': 'EPSG:32733',
#     'region': aoi_geom
# })

# # starting the process
# export_change_mask.start()

# # tracking the process
# while export_change_mask.active():
#   print('Polling for task (id: {}).'.format(export_change_mask.id))
#   time.sleep(5)

In [38]:
# # exporting to Google Asset
# export_change_mask = ee.batch.Export.image.toAsset(**{
#   'image': changed_areas_2015_2017,
#   'description': 'Export Change Mask map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/change_mask_2015_2017', # TODO: change name here
#   'scale': 4.77,
#   'crs': 'EPSG:32733',
#   'region': aoi_geom
# })

# # starting the process
# export_change_mask.start()

# # tracking the process
# while export_change_mask.active():
#   print('Polling for task (id: {}).'.format(export_change_mask.id))

In [39]:
# # exporting to Google Asset
# export_change_mask = ee.batch.Export.image.toAsset(**{
#   'image': changed_areas_2017_2019,
#   'description': 'Export Change Mask map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/change_mask_2017_2019_test', # TODO: change name here
#   'scale': 4.77,
#   'crs': 'EPSG:32733',
#   'region': aoi_geom
# })

# # starting the process
# export_change_mask.start()

# # tracking the process
# while export_change_mask.active():
#   print('Polling for task (id: {}).'.format(export_change_mask.id))

In [40]:
# # exporting to Google Asset
# export_change_mask = ee.batch.Export.image.toAsset(**{
#   'image': changed_areas_2019_2021,
#   'description': 'Export Change Mask map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/test/change_mask_2019_2021', # TODO: change name here
#   'scale': 4.77,
#   'crs': 'EPSG:32733',
#   'region': aoi_geom
# })

# # starting the process
# export_change_mask.start()

# # tracking the process
# while export_change_mask.active():
#   print('Polling for task (id: {}).'.format(export_change_mask.id))

In [41]:
# # exporting to Google Asset
# export_change_mask = ee.batch.Export.image.toAsset(**{
#   'image': changed_areas_2015_2021,
#   'description': 'Export Change Mask map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/change_mask_2015_2021', # TODO: change name here
#   'scale': 4.77,
#   'crs': 'EPSG:32733',
#   'region': aoi_geom
# })

# # starting the process
# export_change_mask.start()

# # tracking the process
# while export_change_mask.active():
#   print('Polling for task (id: {}).'.format(export_change_mask.id))

##### Resources for code snippets

https://colab.research.google.com/github/csaybar/EEwPython/blob/dev/10_Export.ipynb \
https://worldbank.github.io/OpenNightLights/tutorials/mod6_6_RF_classifier.html \
https://towardsdatascience.com/how-to-easily-create-tables-in-python-2eaea447d8fd \
https://developers.google.com/earth-engine/apidocs/ee-classifier-smilerandomforest